In [9]:
# 构造一个sparseTensor
import tensorflow as tf
tf.reset_default_graph()
sparse_tensor=tf.sparse.SparseTensor(indices=[[0, 0], [1, 2]], values=[1, 2], dense_shape=[3, 4])
dense_tensor=tf.constant([[1, 0, 0, 0],
 [0, 0, 2, 0],
 [0, 0, 0, 0]])

with tf.Session() as sess:
    print(sess.run(sparse_tensor))


SparseTensorValue(indices=array([[0, 0],
       [1, 2]]), values=array([1, 2], dtype=int32), dense_shape=array([3, 4]))
[[1 0 0 0]
 [0 0 2 0]
 [0 0 0 0]]


In [16]:
import tensorflow as tf
tf.reset_default_graph()
# sparse和tensor互转

dense_tensor=tf.constant([[1, 0, 0, 0],
 [0, 0, 2, 0],
 [0, 0, 0, 0]])
arr_idx = tf.where(tf.not_equal(dense_tensor, 0))
# gather_nd这个操作就是根据arr_idx的指示，从dense_tensor提取值，最终的结果shape由arr_idx决定
# 而arr_idx的0轴的每个元素都是一个坐标，因此arr_values就是一个一维度向量
arr_values=tf.gather_nd(dense_tensor, arr_idx)
sparse_tensor = tf.SparseTensor(arr_idx,arr_values,dense_shape=dense_tensor.shape)

with tf.Session() as sess:
    print(sess.run(arr_idx))
    print(sess.run(arr_values))
    print(sess.run(sparse_tensor))
    print(sess.run(tf.sparse.to_dense(sparse_tensor)))

[[0 0]
 [1 2]]
[1 2]
SparseTensorValue(indices=array([[0, 0],
       [1, 2]]), values=array([1, 2], dtype=int32), dense_shape=array([3, 4]))
[[1 0 0 0]
 [0 0 2 0]
 [0 0 0 0]]


In [4]:
# 但是不能将一个shape为(None,10)的tensor转化为sparse Tensor
# 形状必须是明确的
import tensorflow as tf
import numpy as np

ds=tf.data.Dataset.from_tensor_slices(np.random.choice([0,1],size=[10,10])).batch(5)

def dense_to_sparse(dense_tensor):
    idx = tf.where(tf.not_equal(dense_tensor, 0))
    values=tf.gather_nd(dense_tensor, idx)
    # 如果不设定形状，那么当dense转sparse的时候会报错的
    # 报错的核心在于dense_shape这个地方
    # dense_shape=dense_tensor.shape 修改为下面这样就可以了
    dense_shape=[5,10]
    return tf.SparseTensor(idx,values,dense_shape=dense_shape)

batch = ds.make_one_shot_iterator().get_next()
sparse_batch=dense_to_sparse(batch)

with tf.Session() as sess:
    print(sess.run(sparse_batch))

SparseTensorValue(indices=array([[0, 0],
       [0, 2],
       [0, 3],
       [1, 0],
       [1, 1],
       [1, 3],
       [1, 4],
       [1, 6],
       [1, 7],
       [1, 9],
       [2, 0],
       [2, 2],
       [2, 3],
       [2, 4],
       [2, 6],
       [2, 8],
       [2, 9],
       [3, 0],
       [3, 7],
       [4, 2],
       [4, 4],
       [4, 5],
       [4, 6],
       [4, 7],
       [4, 8],
       [4, 9]]), values=array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1]), dense_shape=array([ 5, 10]))
